In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os

In [5]:
path_to_data = '/home/dafne/shared/FilterBubble/INCA/'

data = pd.read_csv(os.path.join(path_to_data, 'INCA_klimaat.csv'), index_col=0)

In [20]:
data.shape

(239, 4)

## Topic modeling

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation, NMF

In [8]:
corpus = data['text']

In [10]:
vect = CountVectorizer(min_df=5, max_df=0.9)
tfidf = TfidfTransformer()

In [11]:
X = vect.fit_transform(corpus)
X_tfidf = tfidf.fit_transform(X)

In [ ]:
tf_feature_names = vect.get_feature_names()

In [23]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [29]:
nr_topics = 10

In [30]:
lda = LatentDirichletAllocation(n_components=nr_topics, random_state=0, max_iter=5,
                                learning_method='online',
                                learning_offset=50.).fit(X_tfidf)

In [31]:
print_top_words(lda, tf_feature_names, 10)

Topic #0: vertraging verliest nodig voorziet volgende blijven ipcc kleur kuub denken
Topic #1: dat rutte echter vormen olie eventuele vallen duizend ze niet
Topic #2: zuid afgelopen 500 zorg broeikasgassen zogeheten energietransitie ooit dus hele
Topic #3: wil deden haag noordzee om al zetels niet weten heffing
Topic #4: nooit stand energietransitie dat thunberg meer zich stuk hebben zweedse
Topic #5: dat foto donderdag zijn trump om verkiezingen euro ook land
Topic #6: dat zijn die niet er ook om als aan maar
Topic #7: weleens veroorzaakt speciale uitkomst regenval extreme protesten willen huidige me
Topic #8: november reis reden miljard konden handelsoorlog vindt ontdekten urgenda ondertekend
Topic #9: dat hij die we vergeleken zelfs halen als opgenomen samen



In [32]:
nmf = NMF(n_components=nr_topics, random_state=1,  alpha=.1, l1_ratio=.5).fit(X_tfidf)
print_top_words(nmf, tf_feature_names, 10)

Topic #0: dat die zijn er niet ook als maar aan om
Topic #1: co2 urgenda pbl kabinet 2020 wiebes maatregelen uitstoot klimaatakkoord planbureau
Topic #2: foto afp anp selecteert springende beelden oog willem nl hoofdstad
Topic #3: je ik ben mijn eten wat heb we vlees niet
Topic #4: partij fvd vvd pvda partijen coalitie zetels cda verkiezingen groenlinks
Topic #5: gesprek youth for scholieren climate rutte klimaatspijbelaars jongeren ze premier
Topic #6: gaswinning groningen wiebes gas kuub minister miljard economische gewonnen zaken
Topic #7: ijs zee groenland noordpool opwarming ijskap antarctica smelt gletsjers rond
Topic #8: trump president amerikaanse donald franse macron hij zijn iran leiders
Topic #9: consumenten cbs consumptie punten januari vertrouwen economie twaalf huishoudens februari



In [33]:
X_topics = nmf.transform(X_tfidf)

In [40]:
data['topic'] = np.argmax(X_topics, axis=1)

In [42]:
data[['title', 'topic']]

,title,topic
1,'Lage prijs nieuwe producten hindert circulair...,0
2,Raad van State buigt zich 17 januari na spoedv...,6
3,"Planbureau: Staat haalt klimaatdoelen niet, ui...",1
4,Consumentenvertrouwen flink omlaag: Is dat erg?,9
5,Topman Tata Steel: Staalconcern moet deuren sl...,0
...,...,...
235,Keizerspinguïn zal uitsterven als huidige klim...,0
236,Oudste en dikste Noordpoolijs smelt nu ook,7
237,"Wiebes snapt zorgen over biomassa, maar vindt ...",0
238,Middelbare scholieren Haarlem gaan eigen jonge...,0


In [43]:
data.topic.value_counts()

0    145
4     20
1     15
5     14
6     12
3     12
2      8
8      5
7      5
9      3
Name: topic, dtype: int64